In [20]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
mnist

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000207D53989E8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000207D8107198>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x00000207D812C3C8>)

In [22]:
learning_rate = 0.001
training_iters = 10000
batch_size = 128
display_step = 10

n_inputs = 28
n_steps = 28
n_hidden = 128
n_classes = 10


In [23]:
x = tf.placeholder("float", shape=[None, n_inputs,n_steps])
y = tf.placeholder("float", shape=[None, n_classes])

In [24]:
weights = {
    "out" : tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}

biases = {
    "out" : tf.Variable(tf.random_normal([n_classes]))
         }

In [26]:
def BIRNN(x, weights, biases):
    x = tf.transpose(x, [1, 0, 2]) 
    x = tf.reshape(x, [-1, n_inputs])
    x = tf.split(axis=0, num_or_size_splits=n_steps, value=x)
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']
pred = BIRNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [29]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_steps, n_inputs))
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_inputs))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 2.047282, Training Accuracy= 0.22656
Iter 2560, Minibatch Loss= 1.578247, Training Accuracy= 0.42969
Iter 3840, Minibatch Loss= 1.413420, Training Accuracy= 0.56250
Iter 5120, Minibatch Loss= 1.420945, Training Accuracy= 0.49219
Iter 6400, Minibatch Loss= 0.810249, Training Accuracy= 0.78906
Iter 7680, Minibatch Loss= 1.054210, Training Accuracy= 0.67969
Iter 8960, Minibatch Loss= 0.555927, Training Accuracy= 0.81250
Optimization Finished!
Testing Accuracy: 0.7109375
